In [ ]:
from opto import trace
from opto.optimizers import OptoPrime

import os

import argparse
import concurrent
from dotenv import load_dotenv
from tqdm import tqdm
import textgrad as tg
from textgrad.tasks import load_task
import numpy as np
import random
load_dotenv(override=True)


False

In [2]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

In [3]:
def eval_sample(item, eval_fn, model):
    """
    This function allows us to evaluate if an answer to a question in the prompt is a good answer.

    """
    x, y = item
    x = tg.Variable(x, requires_grad=False, role_description="query to the language model")

    if  np.issubdtype(type(y), np.integer):
        y = int(y)

    y = tg.Variable(y, requires_grad=False, role_description="correct answer for the query")
    response = model(x)
    try:
        eval_output_variable = eval_fn(inputs=dict(prediction=response, ground_truth_answer=y))
        return int(eval_output_variable.value)
    except:
        eval_output_variable = eval_fn([x, y, response])
        eval_output_parsed = eval_fn.parse_output(eval_output_variable)
        return int(eval_output_parsed)

In [4]:
def eval_dataset(test_set, eval_fn, model, max_samples: int=None):
    if max_samples is None:
        max_samples = len(test_set)
    accuracy_list = []

    futures = []
    for _, sample in enumerate(test_set):

        future = eval_sample(sample, eval_fn, model)
        futures.append(future)
        if len(futures) >= max_samples:
            break
    tqdm_loader = tqdm(futures, total=len(futures), position=0)
    for future in tqdm_loader:
        acc_item = future
        accuracy_list.append(acc_item)
        tqdm_loader.set_description(f"Accuracy: {np.mean(accuracy_list)}")
    return accuracy_list
    # with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    #     futures = []
    #     for _, sample in enumerate(test_set):

    #         future = executor.submit(eval_sample, sample, eval_fn, model)
    #         futures.append(future)
    #         if len(futures) >= max_samples:
    #             break
    #     tqdm_loader = tqdm(concurrent.futures.as_completed(futures), total=len(futures), position=0)
    #     for future in tqdm_loader:
    #         acc_item = future.result()
    #         accuracy_list.append(acc_item)
    #         tqdm_loader.set_description(f"Accuracy: {np.mean(accuracy_list)}")
    # return accuracy_list

In [5]:
def run_validation_revert(system_prompt: tg.Variable, results, model, eval_fn, val_set):
    val_performance = np.mean(eval_dataset(val_set, eval_fn, model))
    previous_performance = np.mean(results["validation_acc"][-1])
    print("val_performance: ", val_performance)
    print("previous_performance: ", previous_performance)
    previous_prompt = results["prompt"][-1]

    if val_performance < previous_performance:
        print(f"rejected prompt: {system_prompt.value}")
        system_prompt.set_value(previous_prompt)
        val_performance = previous_performance

    results["validation_acc"].append(val_performance)

In [6]:
set_seed(12)
llm_api_eval = tg.get_engine(engine_name="gpt-4o")
llm_api_test = tg.get_engine(engine_name="gpt-3.5-turbo-0125")
# tg.set_backward_engine(llm_api_eval, override=True)

# Load the data and the evaluation function
train_set, val_set, test_set, eval_fn = load_task("BBH_object_counting", evaluation_api=llm_api_eval)
print("Train/Val/Test Set Lengths: ", len(train_set), len(val_set), len(test_set))
STARTING_SYSTEM_PROMPT = train_set.get_task_description()


Train/Val/Test Set Lengths:  50 100 100


In [7]:
print(STARTING_SYSTEM_PROMPT)


You will answer a reasoning question. Think step by step. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.


In [8]:
train_loader = tg.tasks.DataLoader(train_set, batch_size=3, shuffle=True)


# Testing the 0-shot performance of the evaluation engine
system_prompt = tg.Variable(STARTING_SYSTEM_PROMPT,
                            requires_grad=True,
                            role_description="system prompt to the language model")
model_evaluation = tg.BlackboxLLM(llm_api_eval, system_prompt)

# system_prompt = tg.Variable(STARTING_SYSTEM_PROMPT,
#                             requires_grad=True,
#                             role_description="structured system prompt to a somewhat capable language model that specifies the behavior and strategies for the QA task")
# model = tg.BlackboxLLM(llm_api_test, system_prompt)

# optimizer = tg.TextualGradientDescent(engine=llm_api_eval, parameters=[system_prompt])


## Trace code
@trace.bundle()
def query(system_prompt, *inputs):
    """ Query the language model with the system prompt and the input query """
    return tg.BlackboxLLM(llm_api_test, system_prompt)(*inputs)

system_prompt = trace.node(STARTING_SYSTEM_PROMPT, trainable=True)
optimizer = OptoPrime([system_prompt], prompt_symbols={'variables':'#Parameters'}) # XXX:  to avoid name clash with TextGrad
model = lambda x: query(system_prompt, x).data

results = {"test_acc": [], "prompt": [], "validation_acc": []}
results["test_acc"].append(eval_dataset(test_set, eval_fn, model))
results["validation_acc"].append(eval_dataset(val_set, eval_fn, model))
results["prompt"].append(system_prompt.data)


Accuracy: 0.75: 100%|██████████| 100/100 [00:00<00:00, 1790.99it/s]  


In [9]:
@trace.bundle()
def eval_response(response, ground_truth_answer):
    """ Evaluate the response of the language model with respect to the ground truth answer. 1 means correct, 0 means incorrect """
    try:
        eval_output_variable = eval_fn(inputs=dict(prediction=response, ground_truth_answer=ground_truth_answer))
    except:
        eval_output_variable = eval_fn([x, ground_truth_answer, response])
    return eval_output_variable

@trace.bundle()
def concat(*items):
    """ Concatenate the items into a single string """
    output = ''
    for i, item in enumerate(items):
        output += f'{[i]}: {item}\n\n'
    return output

for epoch in range(3):
    for steps, (batch_x, batch_y) in enumerate((pbar := tqdm(train_loader, position=0))):
        pbar.set_description(f"Training step {steps}. Epoch {epoch}")
        optimizer.zero_feedback()
        feedbacks = []
        for (x, y) in zip(batch_x, batch_y):
            x = tg.Variable(x, requires_grad=False, role_description="query to the language model")

            if  np.issubdtype(type(y), np.integer):
                y = int(y)

            y = tg.Variable(y, requires_grad=False, role_description="correct answer for the query")

            response = query(system_prompt, x)  # node
            eval_output_variable = eval_response(response, y)  # node
            feedbacks.append(eval_output_variable)

        target = concat(*feedbacks)
        print('feedback', [f.data.value for  f in feedbacks])
        target.backward("Improve correctness.")
        optimizer.step(verbose='output')


        tg_system_prompt =tg.Variable(system_prompt.data,
            requires_grad=True,
            role_description="structured system prompt to a somewhat capable language model that specifies the behavior and strategies for the QA task")
        run_validation_revert(tg_system_prompt, results, model, eval_fn, val_set)
        print(" suggested sys prompt: ", system_prompt.data)
        system_prompt._data = tg_system_prompt.value  # to implement the revert

        print("           sys prompt: ", system_prompt.data)
        test_acc = eval_dataset(test_set, eval_fn, model)
        results["test_acc"].append(test_acc)
        results["prompt"].append(system_prompt.data)
        # if steps == 3:
        #     break

Training step 0. Epoch 0: : 0it [00:00, ?it/s]

feedback ['1', '1', '1']
LLM response:
 {
"reasoning": "The instruction is to change parameters to improve output correctness. The code evaluates responses for correctness (1 for correct, 0 for incorrect) based on a string `str0`, which is a prompt used in a language model query. The feedback indicates a need for improvement in correctness, yet the current evaluation outputs suggest the model is already producing correct answers (since `eval_response0`, `eval_response1`, and `eval_response2` are all 1). The parameters do not need alteration because the output shows correct answers for given inputs, contradicting the feedback.",
"answer": "The correctness of the outputs is already satisfactory based on the given inputs.",
"suggestion": {}
}


Accuracy: 0.75: 100%|██████████| 100/100 [00:00<00:00, 1734.42it/s]  

val_performance:  0.75
previous_performance:  0.75
 suggested sys prompt:  You will answer a reasoning question. Think step by step. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.
           sys prompt:  You will answer a reasoning question. Think step by step. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.



Accuracy: 0.83: 100%|██████████| 100/100 [00:00<00:00, 1429.94it/s]  
Training step 1. Epoch 0: : 1it [00:03,  3.86s/it]

...

Accuracy: 0.92: 100%|██████████| 100/100 [00:00<00:00, 1243.03it/s]  
Training step 16. Epoch 2: : 16it [12:38, 40.52s/it]

feedback ['1', '1']
LLM response:
 {
"reasoning": "The feedback indicates a need for improvement in correctness. The current outputs demonstrate that the responses produced by the eval are correct as both eval_response148 and eval_response149 are 1, meaning the answers match the ground truth. The concatenated output concat50 is already '1 1', showing both responses are correct. The given system prompt, found in str0, specifies how to enumerate items and calculate a total. It appears that the outputs are correct as intended, thus the 'correctness' mentioned might refer to some other aspect not apparent from the current context. Without specific incorrect output or contradictions in the feedback, there is no evident change necessary to the parameters.",
"answer": "No apparent changes needed since both responses are already correct.",
"suggestion": {}
}


Accuracy: 0.94: 100%|██████████| 100/100 [00:00<00:00, 2614.82it/s]  


val_performance:  0.94
previous_performance:  0.94
 suggested sys prompt:  You will answer a counting question. List each item and its count explicitly. Then calculate the total. The last line of your response should be: 'Answer: VALUE' where VALUE is a numerical count. Please ensure clarity in explanation.
           sys prompt:  You will answer a counting question. List each item and its count explicitly. Then calculate the total. The last line of your response should be: 'Answer: VALUE' where VALUE is a numerical count. Please ensure clarity in explanation.


Accuracy: 0.92: 100%|██████████| 100/100 [00:00<00:00, 3597.73it/s]  
Training step 16. Epoch 2: : 17it [12:43, 44.92s/it]


In [10]:
accuracies = [np.array(x).mean() for x in results["test_acc"]]
print(accuracies)
print("Best accuracy: ", max(accuracies))

[0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.92, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92]
Best accuracy:  0.92
